In [17]:
import pygame

import numpy as np

import random

In [18]:

render = True
unlock_frame_rate = True
opponent = False

epsilon = 1
epsilonDecay = 0.9995
discount = 0.95
learning_rate = 0.01

resolution = (640, 360) 

accelerationDecay = 0.9995

pygame.init()
screen = pygame.display.set_mode(resolution)
clock = pygame.time.Clock()

paddleHeight = 90


ballRad = 10
ball = pygame.Vector2(screen.get_width() / 2, screen.get_height() / 2)

simpleDifficulty = 2.5

game = 0




In [19]:
def discretize(ball_x, ball_y, paddle_y, vel_x, vel_y):
    discrete_x = int(ball_x / resolution[0]) * q_table_shape[0]



    discrete_y = int((ball_y - paddle_y) / resolution[1] * q_table_shape[1])

    disc_ball_y = int(ball_y / resolution[1] * q_table_shape[1])
    disc_pad_y = int(paddle_y / resolution[1] * q_table_shape[1])


    dis_vel_x = int((vel_x + 10) / 20 * q_table_shape[2])
    dis_vel_y = int((vel_y + 10) / 20 * q_table_shape[3])

    

    return tuple((discrete_x, discrete_y, dis_vel_x, dis_vel_y))

def simpleAI(paddleY, targetY):
    if targetY > paddleY:
        paddleY += simpleDifficulty

    else:
        paddleY -= simpleDifficulty

    return paddleY 

def collision(ball, paddle_rect):
    ball_rect = pygame.Rect(ball.x - ballRad, ball.y - ballRad, ballRad*2, ballRad*2)
    if ball_rect.colliderect(paddle_rect):
        # Collision detected
        return True
    else:
        # No collision detected
        return False

In [20]:


num_actions = 3

q_table_shape = [60, 60, 60, 60]
#q_table = np.random.uniform(low = -2, high = 0, size = (q_table_shape + [num_actions]))
q_table = np.zeros(q_table_shape + [num_actions])


In [21]:
while True:
    game += 1

    acceleration = 0.001
    
    paddle1Y = resolution[1] / 2
    paddle2Y = resolution[1] / 2

    gameOver = False
    if opponent:
        initialVel = 4
    else:
        initialVel = random.uniform(4,7)

    randomStart = random.randint(40, 80)
    randomDir = random.randint(0,1)



    if opponent:
        ballPosX = resolution[0] / 2
        ballPosY = resolution[1] / 2
    else:
        ballPosX = random.randint(10, resolution[0] / 2)
        ballPosY = random.randint(10, resolution[1] - 10)
    if randomDir == 0:
        dirMult = 1
    else:
        dirMult = -1



    ballVelX = (randomStart / 100) * initialVel
    ballVelY = (((initialVel ** 2) - (((randomStart / 100) * initialVel)  ** 2))**0.5) * dirMult

    state = discretize(ballPosX, ballPosY, paddle2Y, ballVelX, ballVelY)
    total_reward = 0
    print('Starting Game, ', game)
    while gameOver != True:
        reward = 0


        # poll for events

        

        # pygame.QUIT event means the user clicked X to close your window
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                gameOver = True

        # fill the screen with a color to wipe away anything from last frame

        paddle1 = pygame.Rect(0.075*resolution[0], paddle1Y - paddleHeight/2, 10, paddleHeight)
        paddle2 = pygame.Rect(0.925*resolution[0], paddle2Y - paddleHeight/2, 10, paddleHeight)

        screen.fill("white")

        pygame.draw.circle(screen, "red", ball, ballRad)
        pygame.draw.rect(screen, 'black', paddle1)
        pygame.draw.rect(screen, 'black', paddle2)


        
        if collision(ball, paddle1):
            ballVelX = -ballVelX
            reward -= 1

        elif collision(ball, paddle2):

            if ballVelX > 0:
                ballVelX = -ballVelX
            reward += 10
            if not opponent:
                gameOver = True
                winner = 1


        if ballPosY < ballRad or ballPosY > resolution[1] - ballRad:
            ballVelY = -ballVelY
        if ballPosX < ballRad:
            winner = 1
            gameOver = True
            reward += 6
            print('CoolAI wins')
        elif ballPosX > resolution[0] - ballRad:
            gameOver = True
            winner = 2
            reward -= 8
            print('BoringAI wins')


        prevHeight = paddle2Y

        



        ballPosX += ballVelX
        ballPosY += ballVelY


        if opponent:
            paddle1Y = simpleAI(paddle1Y, ballPosY)
        

        if random.uniform(0, 1) < epsilon:
            action = random.randint(0, num_actions-1)
        else:
            try:
                action = np.argmax(q_table[state])
            except:
                action = 0        

        

        if action == 1:
            if paddle2Y > 0:

                paddle2Y -= 5
        elif action == 2:
            if paddle2Y < resolution[1]:

                paddle2Y += 5
        else:
            paddle2Y += 0

        

            
        
        new_state = discretize(ballPosX, ballPosY, paddle2Y, ballVelX, ballVelY)
        
        if not gameOver:
            try:
                max_future_q = np.max(q_table[new_state])

                current_q = q_table[state + (action, )]

                new_q = (1 - learning_rate) * current_q + learning_rate * (reward + discount * max_future_q)

                q_table[state + (action, )] = new_q
            except:
                pass
        

        state = new_state

        ball.xy = ballPosX, ballPosY

        
            

        



        if render:
            pygame.display.flip()
        if render and not unlock_frame_rate:
            clock.tick(90)


        ballVelX = ballVelX + acceleration * ballVelX
        ballVelY = ballVelY + acceleration * ballVelY
        acceleration *= accelerationDecay
        total_reward += reward

    if opponent:
        if winner == 1:
            simpleDifficulty += 0.2
        else:
            if simpleDifficulty > 2:
                simpleDifficulty -= 0.1


        print('The simpleAI difficulty is now:', simpleDifficulty)

    print('The AI scored ', total_reward, 'epsilon is', epsilon)



    epsilon *= epsilonDecay




    

        

pygame.quit()

Starting Game,  1
BoringAI wins
The AI scored  -8 epsilon is 1
Starting Game,  2
The AI scored  10 epsilon is 0.9995
Starting Game,  3
The AI scored  10 epsilon is 0.9990002500000001
Starting Game,  4
The AI scored  10 epsilon is 0.9985007498750001
Starting Game,  5
The AI scored  10 epsilon is 0.9980014995000627
Starting Game,  6
The AI scored  10 epsilon is 0.9975024987503127
Starting Game,  7
The AI scored  10 epsilon is 0.9970037475009376
Starting Game,  8
BoringAI wins
The AI scored  -8 epsilon is 0.9965052456271871
Starting Game,  9
The AI scored  10 epsilon is 0.9960069930043736
Starting Game,  10
BoringAI wins
The AI scored  -8 epsilon is 0.9955089895078715
Starting Game,  11
The AI scored  10 epsilon is 0.9950112350131176
Starting Game,  12
The AI scored  10 epsilon is 0.9945137293956111
Starting Game,  13
The AI scored  10 epsilon is 0.9940164725309134
Starting Game,  14
The AI scored  10 epsilon is 0.993519464294648
Starting Game,  15
CoolAI wins
The AI scored  5 epsilon is 

KeyboardInterrupt: 